In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators
using LightXML

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/DataStructures.ji for module DataStructures.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/StatsFuns.ji for module StatsFuns.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/Distances.ji for module Distances.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/NearestNeighbors.ji for module NearestNeighbors.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/Devectorize.ji for module Devectorize.
  likely near /home/ubuntu/.julia/v0.5/Devectorize/src/texpr.jl:293
  likely near /home/ubuntu/.julia/v0.5/Devectorize/src/texpr.jl:293
  likely near /home/ubuntu/.julia/v0.5/NearestNeighbors/src/knn.jl:1
  likely near /home/

In [4]:
const WN_PATH="data/corpora/WordNet-2.1/"
db = DB(WN_PATH)

WordNet.DB

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [3]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");
ee = restore("models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_50__m170000000.model");

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in _collect(::Array{SubString{String},1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Base.EltypeUnknown, ::Base.HasShape) at ./array.jl:283
 in #error_content#12(::Symbol, ::String, ::Function, ::LoadError, ::Array{Ptr{Void},1}) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:75
 in execute_request_0x535c5df2(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:238
 in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:143
 in (::IJulia.##24#30)() at ./task.jl:309
while loading /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl, in expression starting on line 31
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in collect_to!(::Array{String,1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Int64, ::Int64) at ./array.jl:302
 in _collect(:

synthesize_embedding (generic function with 1 method)

In [6]:
function poormans_tokenize(source)
    cleaned = filter(s->!ispunct(s), lowercase(source))
    map(string,  split(cleaned))::Vector{SubString{String}}
end

function punctuation_space_tokenize(source)
    preprocced = lowercase(source)
    pass1=replace(preprocced,r"[[:punct:]]*[[:space:]][[:punct:]]*"," ")
    pass2=replace(pass1,r"[[:punct:]]*$|^[[:punct:]]*","")
    split(pass2)
end



punctuation_space_tokenize (generic function with 1 method)

In [7]:
function load_sense_key_map(wn_basedir)
    path=joinpath(wn_basedir, "dict","index.sense")
    sense_key_map = Dict{Tuple{Int64,String},String}()
    for line in eachline(path)
        full_sense_key, offset_str, sense_num_str, tagcount_str = split(line) 
        lemma_name = first(split(full_sense_key,'%'))
        sense_offset = parse(Int64, offset_str)
        index = (sense_offset,lemma_name)
        @assert(!haskey(sense_key_map, index))
        sense_key_map[index] = full_sense_key
    end
    sense_key_map
end


function sense_key(lem::Lemma, ss::Synset,sense_key_map)
    sense_key_map[(ss.offset,lem.word)]
end
function sense_keys(lem::Lemma,sense_key_map)
    [sense_key_map[(ss_offset,lem.word)] for ss_offset in lem.synset_offsets ]
end

sense_keys (generic function with 1 method)

In [8]:
sense_key_map = load_sense_key_map(WN_PATH);

In [11]:
sense_keys(db['a',"six"],sense_key_map)

1-element Array{String,1}:
 "six%5:00:00:cardinal:00"

In [12]:
sense_key_map[2134693,"discover"]

"discover%2:39:03::"

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [ ]:
immutable WsdChallenge
    id::String
    word::String
    lemma::String
    pos::Char
    context::Vector{String}
end


In [17]:
function load_challenges_semeval2007(xml_file="data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml")
    xdoc = parse_file(xml_file)
    xroot = root(xdoc)
    Task() do
        for text_node in child_elements(xroot)
            #@show text_node
            #text = child_nodes(text_node) |> collect
            #println(text)
            for sentence_node in child_elements(text_node)
                sentence = punctuation_space_tokenize(content(sentence_node))

                for lemma_node in child_elements(sentence_node)
                    word = content(lemma_node) |> lowercase
                    lemma = attribute(lemma_node,"lemma")|> lowercase
                    pos = first(attribute(lemma_node,"pos"))
                    id = attribute(lemma_node,"id")
                    context = filter(w->w!=word, sentence)
                    produce(WsdChallenge(id, word, lemma, pos, context))
                end
            end
        end
    end
end
challenges = load_challenges_semeval2007() |> collect;

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:39 [inlined]
 in content(::LightXML.XMLNode) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:171
 in (::##15#17{LightXML.XMLElement})() at ./In[17]:18
while loading In[17], in expression starting on line 32
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:39 [inlined]
 in #attribute#1(::Bool, ::Function, ::LightXML.XMLElement, ::String) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:226
 in (::##15#17{LightXML.XMLElement})() at ./In[17]:22
while loading In[17], in expression starting on line 32


In [132]:
immutable WsdSolution
    id
    lemma
    pos
    solutions
end
only_of_pos(data, pos) = filter(d->d.pos==pos, data)

function load_solutions(key_file="data/corpora/wsd/semeval2007_t7/key/dataset21.test.key")
    map(eachline(key_file)) do line
        line_data, comment = split(line,"!!")
        fields = split(line_data)
        doc_id = fields[1]
        instance_id = fields[2]
        solutions = fields[3:end]
        
        lemma,pos = match(r"lemma=(.*)#(.)", comment).captures
        WsdSolution(instance_id, lemma, pos[1], solutions)
    end
end

solutions = load_solutions();

LoadError: LoadError: KeyError: key "in addition" not found
while loading In[143], in expression starting on line 3

In [20]:
function sense_frequency(ss::Synset, lem::Lemma)
    lem_count = lem.tagsense_count
    lem_count+0.1(sum(values(ss.word_counts))-lem_count)
end

sense_frequency (generic function with 1 method)

In [179]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end


function synthesize_embedding(ee,context, phrase)
    words = split(phrase," ")
    wvs = vcat((get(ee.embedding,w,Vector{Float32}[]) for w in words)...)
    if length(wvs) == 0
            throw(KeyError("None of $words have embeddings"))
    end
    logprobs = [Query.logprob_of_context(ee, context,wv; skip_oov=true, normalise_over_length=true) for wv in wvs]
    weighted_average(logprobs, wvs)
end


all_identical(col) = length(col)==1 || !any(x->x!=col[1],col[2:end])

lexically_informed_embeddings(db,ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)
function lexically_informed_embeddings(db,ee, word,lemma_word, pos)
    
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
         
    embeddings = Vector{Vector{Float32}}(length(target_synsets))
    for (ii,synset) in enumerate(target_synsets)
        context::Vector{SubString{String}} = collect(filter!(w->w!=word, punctuation_space_tokenize(synset.gloss)))
        embeddings[ii] = synthesize_embedding(ee,context, word)
    end
        
    
    if all_identical(embeddings)
        #Use MCWS
        score, index = findmax(sense_frequency(ss,lemma) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
        embeddings=[embeddings[1]]
    end
        
    embeddings,target_synsets,lemma
end

lexically_informed_embeddings (generic function with 2 methods)

In [180]:
mw_challenges = [c for c in challenges if contains(c.lemma,"_")]
challenge = mw_challenges[1]
lexically_informed_embeddings(db,ee, challenge.word, challenge.lemma, challenge.pos)


(Array{Float32,1}[Float32[-0.0835615,0.0436134,0.0357565,-0.195218,-0.0150616,0.0623249,0.000887029,0.0132962,-0.0190318,-0.0587051,-0.191609,0.0257554,0.0670426,-0.00430257,-0.0481629,0.0370439,0.0283803,-0.0854695,-0.0418194,0.0370742,0.105485,-0.0919684,0.0572593,0.0477335,0.0413913,-0.018515,0.0389618,0.318222,-0.0848608,-0.0312266,0.0250157,0.037267,0.0501295,0.0405783,0.0541011,-0.160077,0.082792,0.154932,-0.151211,0.0877451,-0.13094,-0.0204533,-0.0477487,0.00436434,0.040013,0.0844666,0.00706025,0.0455183,0.122014,-0.102058]],WordNet.Synset[(r) in addition, additionally, to boot (by way of addition; furthermore; "he serves additionally as the CEO")],in_addition.r)

In [181]:
function supervised_wsd(challenge::WsdChallenge, ee::WordSenseEmbedding, wn::DB, sense_key_map)
    try
        embeddings,target_synsets,lemma = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        #sense_index = Query.WSD(ee, embeddings, challenge.context; skip_oov=true)
        logprobs_of_context = [logprob_of_context(ee,challenge.context, wv; skip_oov=true, normalise_over_length=true) 
                                    for wv in embeddings] 
      
        sense_index= indmax(logprobs_of_context)

        
        synset = target_synsets[sense_index]
        sk = sense_key(lemma, synset, sense_key_map)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end

function most_frequent_sense(challenge, wn::DB, sense_key_map)
    try
        lemma = wn[challenge.pos, challenge.lemma]
        target_synsets::Vector{Synset} = synsets(db, lemma)
        
        sense_freqs =  Float32[sense_frequency(ss,lemma) for ss in target_synsets]      
        sense_index= indmax(sense_freqs)
        synset = target_synsets[sense_index]
        
        sk = sense_key(lemma, synset, sense_key_map)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end



most_frequent_sense (generic function with 3 methods)

In [182]:
function evalute_on_wsd_challenge(challenges, solutions, method)
    correct = 0
    incorrect = 0
    notattempted = 0
    for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = method(challenge)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end
    
    
function evalute_on_wsd_challenge(challenges, solutions, ee::WordSenseEmbedding, wn::DB, sense_key_map)
    method = challenge -> supervised_wsd(challenge, ee, wn, sense_key_map)
    evalute_on_wsd_challenge(challenges, solutions, method)
end


function evalute_on_wsd_challenge_MFS(challenges, solutions, wn::DB, sense_key_map)
    method = challenge -> most_frequent_sense(challenge, wn, sense_key_map)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

evalute_on_wsd_challenge_MFS (generic function with 1 method)

In [183]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db, sense_key_map))


overall: 		(0.6370501999111506,0.6319964742177171,0.6345132743362832)
only nouns  : 		(0.6554545454545454,0.6507220216606499,0.6530797101449276)
only verbs  : 		(0.5502555366269165,0.5465313028764806,0.5483870967741935)
only adjecti: 		(0.7011173184357542,0.6933701657458563,0.6972222222222222)
only adverbs: 		(0.6747572815533981,0.6682692307692307,0.6714975845410628)


In [42]:
println("overall: \t\t", evalute_on_wsd_challenge_MFS(challenges, solutions, db, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), db, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), db, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), db, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), db, sense_key_map))


overall: 		(0.5932128691053328,0.5932128691053328,0.5932128691053328)
only nouns  : 		(0.592057761732852,0.592057761732852,0.592057761732852)
only verbs  : 		(0.48392554991539766,0.48392554991539766,0.48392554991539766)
only adjecti: 		(0.6823204419889503,0.6823204419889503,0.6823204419889503)
only adverbs: 		(0.7548076923076923,0.7548076923076923,0.7548076923076922)


In [64]:
#Zero shot WSI for 1 shot WSD
using Training


Use "Dict(a=>b for (a,b) in c)" instead.


In [187]:
rr = FixedWordSenseEmbedding(ee.dimension, random_inited, huffman_tree; initial_nsenses=1)
rr.distribution = ee.distribution
rr.codebook = ee.codebook
rr.classification_tree = ee.classification_tree
Training.initialize_embedding(rr)

In [190]:
rr.initial_nsenses=100
Training.initialize_embedding(rr)

In [191]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, rr, db, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), rr, db, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), rr, db, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), rr, db, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), rr, db, sense_key_map))


overall: 		(0.5886272767658818,0.5839576906126047,0.5862831858407079)
only nouns  : 		(0.5881818181818181,0.5839350180505415,0.586050724637681)
only verbs  : 		(0.4787052810902896,0.4754653130287648,0.4770797962648557)
only adjecti: 		(0.6787709497206704,0.6712707182320442,0.675)
only adverbs: 		(0.7475728155339806,0.7403846153846154,0.7439613526570048)


In [196]:
hh = deepcopy(rr)
for word in keys(rr.embedding)
    append!(hh.embedding[word], ee.embedding[word])
end

In [197]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, hh, db, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), hh, db, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), hh, db, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), hh, db, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), hh, db, sense_key_map))

overall: 		(0.6126166148378498,0.6077567210224769,0.6101769911504425)
only nouns  : 		(0.6263636363636363,0.621841155234657,0.6240942028985507)
only verbs  : 		(0.5434412265758092,0.5397631133671743,0.5415959252971138)
only adjecti: 		(0.6731843575418994,0.6657458563535912,0.6694444444444444)
only adverbs: 		(0.6310679611650486,0.625,0.6280193236714976)
